In [50]:
import json
import pandas as pd
import csv
import numpy
import requests
from pprint import pprint

api_key = "a2rOTe8PfBiJBwTOSlRbteARqAEa0s6DsRyoSOOF"
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json?'

search = 'school.degrees_awarded.predominant=3&_fields=id,school.name,latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,school.region_id,latest.earnings.10_yrs_after_entry.median,latest.earnings.6_yrs_after_entry.median,latest.repayment.5_yr_repayment.completers_rate,latest.repayment.7_yr_repayment.completers_rate,latest.repayment.3_yr_repayment.completers_rate'

search_url = url + search + '&api_key=' + api_key

response = requests.get(search_url)
response_json = response.json()
# pprint (response.json())


{'metadata': {'page': 0, 'per_page': 20, 'total': 2113},
 'results': [{'id': 445188,
              'latest.cost.tuition.in_state': 13262,
              'latest.cost.tuition.out_of_state': 39944,
              'latest.earnings.10_yrs_after_entry.median': None,
              'latest.earnings.6_yrs_after_entry.median': 36900,
              'latest.repayment.3_yr_repayment.completers_rate': 0.7814726841,
              'latest.repayment.5_yr_repayment.completers_rate': 0.786069652,
              'latest.repayment.7_yr_repayment.completers_rate': None,
              'school.name': 'University of California-Merced',
              'school.region_id': 8},
             {'id': 446792,
              'latest.cost.tuition.in_state': None,
              'latest.cost.tuition.out_of_state': None,
              'latest.earnings.10_yrs_after_entry.median': 29300,
              'latest.earnings.6_yrs_after_entry.median': 23500,
              'latest.repayment.3_yr_repayment.completers_rate': 0.347467609,


In [47]:
# school_ids_list = []
# schools_list = []
# regions_list = []

# for page in range (0,105):
#     search_url = f'{url}{search}&page={page}&api_key={api_key}'
#     response = requests.get(search_url)
#     response_json = response.json()
       
#     for num in range(0,20):
#         school_id = response_json['results'][num]['id']
#         school_ids_list.append(school_id)
#         school_name = response_json['results'][num]['school.name']
#         schools_list.append(school_name)  
#         region_id = response_json['results'][num]['school.region_id']
#         regions_list.append(region_id)
                
# # print (len(school_ids_list))
# # print (len(schools_list))
# # print (len(regions_list))

# schoolinfo_df = pd.DataFrame({
#     "School ID": school_ids_list,
#     "School": schools_list,
#     "Region ID": regions_list
# })

# schoolinfo_df.to_csv("schoolinfo.csv", index=False, header=False)
# schoolinfo_df                    
            


KeyError: 'results'

In [52]:
# school_ids_list = []
# tuition_in_state_list = []
# tuition_out_state_list = []

# for page in range (0,105):
#         search_url = f'{url}{search}&page={page}&api_key={api_key}'
#         response = requests.get(search_url)
#         response_json = response.json()
        
#         for num in range(0,20):
#             school_id = response_json['results'][num]['id']
#             school_ids_list.append(school_id)                
#             tuition_in = response_json['results'][num]['latest.cost.tuition.in_state']
#             tuition_in_state_list.append(tuition_in)  
#             tuition_out = response_json['results'][num]['latest.cost.tuition.out_of_state']
#             tuition_out_state_list.append(tuition_out)
                
# school_tuition_df = pd.DataFrame({
#     "School ID": school_ids_list,
#     "Tuition In of State": tuition_in_state_list,
#     "Tuition Out of State": tuition_out_state_list
# })
                
# school_tuition_df.to_csv("school_tuition.csv", index=False, header=False)
# school_tuition_df                

In [54]:
csvpath = 

,School ID,Tuition In of State,Tuition Out of State
0,445188,13262.0,39944.0
1,446792,NaN,NaN
2,446808,NaN,NaN
3,446048,19128.0,19128.0
4,446668,NaN,NaN
5,447458,23899.0,23899.0
6,447272,13152.0,13152.0
7,117557,10575.0,10575.0
8,210492,19932.0,19932.0
9,441982,49986.0,49986.0
